In [90]:
import pandas as pd
import joblib

df = pd.read_csv("호선정리.csv", encoding='CP949')
list_data = df.values.tolist()

def shortest_path_2ho(start_idx, end_idx, stations):
    direct_path = abs(end_idx - start_idx)
    if direct_path <= len(stations) / 2:
        if start_idx < end_idx:
            return start_idx , end_idx , "상선"
        else :
            start_idx, end_idx = end_idx + 1, start_idx + 1
            return start_idx , end_idx , "하선"

    else:
        if start_idx < end_idx:
            start_idx,end_idx = start_idx + 1, end_idx + 1
            return end_idx, start_idx + len(stations), "하선"
        else: 
            return start_idx, end_idx + len(stations), "상선"

def print_congestion_predictions(start_station, end_station, input_YEAR, input_MONTH, input_DAY, input_LINE, input_TIME, station_data):
    stations = [item[2] for item in station_data if item[1] == input_LINE]
    start_idx, end_idx = stations.index(start_station), stations.index(end_station)
    
    is_line_2 = input_LINE == 2
    if is_line_2:
        start_idx, end_idx, direction = shortest_path_2ho(start_idx, end_idx, stations)
    else:
        direction = "상선" if start_idx < end_idx else "하선"
        if direction == "하선":
            start_idx, end_idx = end_idx + 1 , start_idx + 1

    if input_LINE in [21,22]:
        input_LINE = 2

    if input_LINE in [51]:
        input_LINE = 5

    model = joblib.load(f'XGB_reg_num{input_LINE}.pkl')

    input_datas = []
    for idx in range(start_idx , end_idx):
        station = stations[idx % len(stations)]
        input_datas.append({
            'YEAR': input_YEAR,
            'MONTH': input_MONTH,
            'DAY': input_DAY,
            'STATION': station,
            'DIRECTION': direction,
            'TIME_00': input_TIME
        })

    df_input = pd.DataFrame(input_datas)
    predictions = model.predict(df_input)

    for station, pred in zip(df_input['STATION'], predictions):
        print(f"역: {station}, 예측 혼잡도: {pred:.2f}")
    print(direction)
    avg_congestion = predictions.mean()
    print(f"평균 혼잡도: {avg_congestion:.2f}")

In [91]:
print_congestion_predictions('강남', '이대', 2023, 5, '월', 2, 'TIME_18', list_data)

역: 강남, 예측 혼잡도: 133.54
역: 교대, 예측 혼잡도: 155.76
역: 서초, 예측 혼잡도: 157.68
역: 방배, 예측 혼잡도: 163.89
역: 사당, 예측 혼잡도: 124.08
역: 낙성대, 예측 혼잡도: 120.11
역: 서울대입구, 예측 혼잡도: 110.27
역: 봉천, 예측 혼잡도: 103.49
역: 신림, 예측 혼잡도: 81.54
역: 신대방, 예측 혼잡도: 84.74
역: 구로디지털단지, 예측 혼잡도: 102.09
역: 대림, 예측 혼잡도: 93.61
역: 신도림, 예측 혼잡도: 83.38
역: 문래, 예측 혼잡도: 79.79
역: 영등포구청, 예측 혼잡도: 77.46
역: 당산, 예측 혼잡도: 76.12
역: 합정, 예측 혼잡도: 59.96
역: 홍대입구, 예측 혼잡도: 47.17
역: 신촌, 예측 혼잡도: 49.93
상선
평균 혼잡도: 100.24
